## Dashboard

#### Utils (just run this cell) 

In [13]:
import pandas as pd
import os
import json
import csv
from datetime import datetime
import yaml


def extract_scores(dashboard_dir, output_csv):
    """
    Traverse the dashboard directory, extract macro avg F1 scores from JSON files,
    and save to a CSV file along with relevant metadata.

    Parameters:
        dashboard_dir (str): Base directory containing the structured dashboard outputs.
        output_csv (str): Path to save the resulting CSV file.
    """
    entries_processed = 0

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)

    # Open the CSV file for writing
    with open(output_csv, mode="w", newline="") as csv_file:
        csv_writer = csv.writer(csv_file)

        # Write CSV header
        csv_writer.writerow(
            [
                "timestamp",
                "model_name",
                "layer",
                "width",
                "type",
                "dataset_name",
                "dataset_split",
                "linear_macro_f1_score",
                "linear_accuracy",
                "linear_mean_cv_accuracy",
                "linear_std_cv_accuracy",
                "decision_tree_macro_f1_score",
                "decision_tree_accuracy",
                "decision_tree_mean_cv_accuracy",
                "decision_tree_std_cv_accuracy",
            ]
        )

        # Traverse the dashboard directory
        for root, _, files in os.walk(dashboard_dir):
            for file in files:
                if file.endswith(".json"):
                    json_path = os.path.join(root, file)

                    try:
                        # Load the JSON file
                        with open(json_path, "r") as f:
                            data = json.load(f)

                        # Extract metadata
                        metadata = data.get("metadata", {})
                        models = data.get("models", {})

                        timestamp = metadata.get("timestamp", "N/A")
                        model_name = metadata.get("model", {}).get("name", "N/A")
                        layer = metadata.get("model", {}).get("layer", "N/A")
                        width = metadata.get("args", {}).get("width", "N/A")
                        dataset_name = metadata.get("dataset", {}).get("name", "N/A")
                        dataset_split = metadata.get("args", {}).get(
                            "dataset_split", "N/A"
                        )
                        hidden = metadata.get("dataset", {}).get("hidden", "N/A")

                        if hidden:
                            hidden_value = "Hidden States"
                        else:
                            hidden_value = "SAE Features"

                        # Extract Linear Probe metrics
                        linear_probe = models.get("linearProbe", {})
                        linear_macro_avg = linear_probe.get(
                            "aggregated_metrics", {}
                        ).get("macro avg", {})
                        linear_f1_score = linear_macro_avg.get("f1_score", "N/A")

                        linear_performance = linear_probe.get("performance", {})
                        linear_accuracy = linear_performance.get("accuracy", "N/A")
                        linear_cv = linear_performance.get("cross_validation", {})
                        linear_mean_cv_accuracy = linear_cv.get("mean_accuracy", "N/A")
                        linear_std_cv_accuracy = linear_cv.get("std_accuracy", "N/A")

                        # Extract Decision Tree metrics
                        decision_tree = models.get("decisionTree", {})
                        decision_tree_macro_avg = decision_tree.get(
                            "aggregated_metrics", {}
                        ).get("macro avg", {})
                        decision_tree_f1_score = decision_tree_macro_avg.get(
                            "f1_score", "N/A"
                        )

                        decision_tree_performance = decision_tree.get("performance", {})
                        decision_tree_accuracy = decision_tree_performance.get(
                            "accuracy", "N/A"
                        )
                        decision_tree_cv = decision_tree_performance.get(
                            "cross_validation", {}
                        )
                        decision_tree_mean_cv_accuracy = decision_tree_cv.get(
                            "mean_accuracy", "N/A"
                        )
                        decision_tree_std_cv_accuracy = decision_tree_cv.get(
                            "std_accuracy", "N/A"
                        )

                        # Append extracted data to CSV
                        csv_writer.writerow(
                            [
                                timestamp,
                                model_name,
                                layer,
                                width,
                                hidden_value,
                                dataset_name,
                                dataset_split,
                                linear_f1_score,
                                linear_accuracy,
                                linear_mean_cv_accuracy,
                                linear_std_cv_accuracy,
                                decision_tree_f1_score,
                                decision_tree_accuracy,
                                decision_tree_mean_cv_accuracy,
                                decision_tree_std_cv_accuracy,
                            ]
                        )

                        entries_processed += 1

                    except Exception as e:
                        print(f"Error processing {json_path}: {e}")

    print(f"Processed {entries_processed} entries.")
    print(f"Results saved to {output_csv}")

### Read in all the scores

In [15]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    dashboard_dir = config["settings"]["base_classify_dir"]

output_csv = os.path.join(dashboard_dir, "classify_scores.csv")
extract_scores(dashboard_dir, output_csv)

Processed 420 entries.
Results saved to ./output/classifications/classify_scores.csv


## Plots


Output()